In [1]:
import pandas as pd
import numpy as np
df = pd.read_csv("TKB HKI 2017-2018.csv")
df.head()

,MaMH,TenMH,NhomTo,ToTH,TenLop,TongSoSV,ThuKieuSo,TietBD,SoTiet,MaPH,DSTuanHoc,MaNV,TenDayDuNV,IsTKBDaXep,MaDV,TenDV,TenDVEg,NgayBD,NgayKT
0,1,Functional Programming,1,NaN,"ITIT16UN11, ITIT16UN21",10,2,1,3,LA1.605,-2345678901234567---,174,Dao Tran Hoang Chau,0,IT IT,Computer Science & Engineering,Computer Science & Engineering,42989,43094
1,2,Operating Systems,1,NaN,"ITIT15CS1, ITIT15IU11, ITIT15IU21, ITIT15IU31",90,6,6,3,A1.309,-2345678901234567---,119,Tran Manh Ha,0,IT IT,Computer Science & Engineering,Computer Science & Engineering,42993,43098
2,2,Operating Systems,1,1.0,"ITIT15CS1, ITIT15IU11, ITIT15IU21, ITIT15IU31",30,2,6,4,LA1.605,-----678901234------,119,Tran Manh Ha,0,IT IT,Computer Science & Engineering,Computer Science & Engineering,43017,43073
3,2,Operating Systems,1,2.0,"ITIT15CS1, ITIT15IU11, ITIT15IU21, ITIT15IU31",30,2,1,4,LA1.608,-----678901234------,213,Nguyen Tien Dung,0,IT IT,Computer Science & Engineering,Computer Science & Engineering,43017,43073
4,2,Operating Systems,1,3.0,"ITIT15CS1, ITIT15IU11, ITIT15IU21, ITIT15IU31",30,2,6,4,LA1.607,-----678901234------,213,Nguyen Tien Dung,0,IT IT,Computer Science & Engineering,Computer Science & Engineering,43017,43073


In [2]:
df = pd.DataFrame(df)
df1 = df[['MaMH', 'TenMH', 'ToTH', 'TongSoSV', 'SoTiet', 'MaPH','MaNV', 'TenDayDuNV']]
df1 = df1.rename(columns={'MaMH': 'course_id', 'TenMH': 'course_name','ToTH': 'Lab', 'TongSoSV': 'size', 'SoTiet': 'duration', 'MaPH': 'room', 'MaNV': 'prof_id', 'TenDayDuNV': 'prof_name' })
df1['Lab'] = df1['Lab'].fillna(0)
df1['Lab'] = df1['Lab'].astype(str)
df1['prof_id'] = df1['prof_id'].astype(int)
df1['course_id'] = df1['course_id'].astype(int)

for index, row in df1.iterrows():
    if row['Lab'] == '1.0' or row['Lab'] == '2.0' or row['Lab'] == '3.0' or row['Lab'] == '4.0':
        df1.at[index, 'Lab'] = 'True'
    else:
        df1.at[index, 'Lab'] = ''
        
df1['Lab'] = df1['Lab'].astype(bool)
df1.reset_index(inplace=True)
df1 = df1.rename(columns={'index': 'group_id'})
df1['group_id'] = np.arange(1, len(df) + 1)
df1.head()

,group_id,course_id,course_name,Lab,size,duration,room,prof_id,prof_name
0,1,1,Functional Programming,False,10,3,LA1.605,174,Dao Tran Hoang Chau
1,2,2,Operating Systems,False,90,3,A1.309,119,Tran Manh Ha
2,3,2,Operating Systems,True,30,4,LA1.605,119,Tran Manh Ha
3,4,2,Operating Systems,True,30,4,LA1.608,213,Nguyen Tien Dung
4,5,2,Operating Systems,True,30,4,LA1.607,213,Nguyen Tien Dung


In [3]:
room_default = [['A1.309', 90, 0],
                ['L107', 40, 0],
                ['LA1.605', 35, 1],
                ['LA1.607', 35, 1],
]
room_columns = ['room', 'size', 'Lab']
df_room = pd.DataFrame(room_default, columns=room_columns)
df_room['Lab'] = df_room['Lab'].astype(str)
for index, row in df_room.iterrows():
    if row['Lab'] == '1':
        df_room.at[index, 'Lab'] = 'True'
    else:
        df_room.at[index, 'Lab'] = ''
df_room['Lab'] = df_room['Lab'].astype(bool)
df_room

,room,size,Lab
0,A1.309,90,False
1,L107,40,False
2,LA1.605,35,True
3,LA1.607,35,True


In [4]:
import json

# create list of dictionaries representing each object in the JSON file
objects = []

for index, row in df1.iterrows():
    if row['prof_id'] != '':
        # create professor object
        prof = {
            "prof": {
                "id": row['prof_id'],
                "name": row['prof_name']
            }
        }
        if prof not in objects:
            objects.append(prof)

    if row['course_id'] != '':
        # create course object
        course = {
            "course": {
                "id": row['course_id'],
                "name": row['course_name']
            }
        }
        if course not in objects:
            objects.append(course)

    if row['group_id'] != '':
        # create room object
        group = {
            "group": {
                "id": row['group_id'],
                "size": row['size']
            }
        }
        if group not in objects:
            objects.append(group)
            
    if row['prof_id'] != '' and row['course_id'] != '':
        # create class object
        class_ = {
            "class": {
                "professor": row['prof_id'],
                "course": row['course_id'],
                "duration": row['duration'],
                "group": row['group_id'],
                "lab": row['Lab']
            }
        }
        if class_ not in objects:
            objects.append(class_)
        
for index, row in df_room.iterrows():
    if row['room'] != '':
        # create room object
        room = {
            "room": {
                "name": row['room'],
                "lab": row['Lab'],
                "size": row['size']
            }
        }
        objects.append(room)    
        
# create JSON object with list of objects
json_data = json.dumps(objects, sort_keys=False)

# write JSON object to file
with open('GaSchedule3.json', 'w') as f:
    f.write(json_data)

In [5]:
json_data

'[{"prof": {"id": 174, "name": "Dao Tran Hoang Chau"}}, {"course": {"id": 1, "name": "Functional Programming"}}, {"group": {"id": 1, "size": 10}}, {"class": {"professor": 174, "course": 1, "duration": 3, "group": 1, "lab": false}}, {"prof": {"id": 119, "name": "Tran Manh Ha"}}, {"course": {"id": 2, "name": "Operating Systems"}}, {"group": {"id": 2, "size": 90}}, {"class": {"professor": 119, "course": 2, "duration": 3, "group": 2, "lab": false}}, {"group": {"id": 3, "size": 30}}, {"class": {"professor": 119, "course": 2, "duration": 4, "group": 3, "lab": true}}, {"prof": {"id": 213, "name": "Nguyen Tien Dung"}}, {"group": {"id": 4, "size": 30}}, {"class": {"professor": 213, "course": 2, "duration": 4, "group": 4, "lab": true}}, {"group": {"id": 5, "size": 30}}, {"class": {"professor": 213, "course": 2, "duration": 4, "group": 5, "lab": true}}, {"prof": {"id": 74, "name": "Nguyen Van Sinh"}}, {"course": {"id": 3, "name": "IT Project Management"}}, {"group": {"id": 6, "size": 35}}, {"clas